In [2]:
from flask import Flask, request, jsonify
import pickle
import datetime
import pandas as pd
from sqlalchemy import create_engine
import os
os.getcwd()
import sqlite3

In [5]:
app = Flask(__name__)

# Cargar el modelo entrenado
with open("titanic_model.pkl", "rb") as f:
    model = pickle.load(f)

In [4]:
# Conectar a la base de datos (se crea el archivo si no existe)
def init_db():
    connection = sqlite3.connect("predictions.db")  
    crsr = connection.cursor()
    query  = '''
        CREATE TABLE IF NOT EXISTS predictions (
            Pclass INT,
            Sex INT,
            Age FLOAT,
            prediction INT,
            timestamp TIMESTAMP
    );
'''
    crsr.execute(query)

init_db()

In [6]:
@app.route('/predict', methods=['POST'])
def predict():
    """
    Endpoint que recibe datos en formato JSON, realiza una predicción y guarda los datos en la base de datos.
    """
    try:
        data = request.json
        # 1. Extraer los datos de entrada del JSON recibido
        Pclass = data["Pclass"]
        Sex= data["Sex"]
        Age = data["Age"]
        input_data = [[Pclass, Sex, Age]]
        # 2. Realizar predicción con el modelo
        prediction = model.predict(input_data)[0]

        # 3. Guardar en la base de datos
        timestamp = datetime.datetime.now().isoformat()
        connection = sqlite3.connect("predictions.db")
        crsr = connection.cursor()
        # Completa aquí: inserta los datos (inputs, predicción, timestamp) en la base de datos
        query = '''
                INSERT INTO predictions (Pclass, Sex, Age, prediction, timestamp)
                VALUES (Pclass, Sex, Age, prediction, timestamp);
                '''
        crsr.execute(query, (Pclass, Sex, Age, int(prediction), timestamp)
)
        return jsonify({"prediction": int(prediction), "timestamp": timestamp})
    except Exception as e:
        return jsonify({"error": str(e)})